In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ca = pd.read_csv('../data/Initiated Building Permits (CA Tasks) 3yr.csv',
                 parse_dates=['Date_Initiated','Date_Issued','Task_Complete_Date']) 
fr = pd.read_csv('../data/Initiated Building Permits (FR Tasks) 3yr.csv',
                 parse_dates=['Date_Initiated','Date_Issued','Task_Complete_Date']) 
pl = pd.read_csv('../data/Initiated Building Permits (PL Tasks) 3yr.csv',
                 parse_dates=['Date_Initiated','Date_Issued','Task_Complete_Date']) 
pw = pd.read_csv('../data/Initiated Building Permits (PW Tasks) 3yr.csv', 
                 parse_dates=['Date_Initiated','Date_Issued','Task_Complete_Date']) 
sw = pd.read_csv('../data/Initiated Building Permits (SW Tasks) 3yr.csv', 
                 parse_dates=['Date_Initiated','Date_Issued','Task_Complete_Date']) 
ws = pd.read_csv('../data/Initiated Building Permits (WS Tasks) 3yr.csv', 
                 parse_dates=['Date_Initiated','Date_Issued','Task_Complete_Date']) 
u_o = pd.read_csv('../data/Initiated Building Permits 3yr (Temp-Final U_O).csv', 
                  parse_dates=['Date_Initiated','Date_Issued','Temp U&O Complete Date','Final U&O Complete Date']) 

C:\Users\pfbab\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
list_dept = {'fr':fr,'pl':pl,'pw':pw,'sw':sw,'ws':ws,'ca':ca}
convert_dict = {'CASE_TYPE': 'category',
                'SUB_TYPE': 'category',
                'STATUS_CODE': 'category',
                'RESULT_CODE': 'category'
               }
all_depts_app = pd.DataFrame()
all_depts_inspect = pd.DataFrame()
task_filter = {}
all_pivot = pd.DataFrame()

In [4]:
for key in list_dept:
    df = list_dept[key]
    df['dept'] = str(key)
    df = df[df['PERMIT #'].astype(str).str.isnumeric()] # T&D filter
    df = df.astype(convert_dict)
    df['deltatask'] = df['Task_Complete_Date'] - df['Date_Initiated'] #create column for task length
    df = df.astype({'deltatask':"timedelta64[D]"}) #make timedelta work for pivot table
    # find tasks completed after permit issued
    df['postissuedtask'] = df['Task_Complete_Date'] > df['Date_Issued']
    post_taskcode = df[['TASK_DESC','postissuedtask']].groupby(['TASK_DESC']).agg(['sum', 'count'])
    filter_posttaskcode = post_taskcode[post_taskcode.iloc[:,0] >= post_taskcode.iloc[:,1]*.95].reset_index()
    #create list of tasks for application and inspection 
    df_inspect = df[df['TASK_DESC'].isin(filter_posttaskcode.TASK_DESC)]
    df_app = df[~df['TASK_DESC'].isin(filter_posttaskcode.TASK_DESC)]
    #keep only the last task completed
    df_last = df_app.sort_values(by=['Task_Complete_Date','RESULT_DESC'], ascending=[False,True]).drop_duplicates(subset=['PERMIT #', 'TASK_CODE'], keep='first')
    #pivot data to aggregate
    df_pivot = pd.pivot_table(df_last, values='deltatask', index=['CASE_TYPE'], aggfunc=[np.mean, np.median, np.min, np.max, np.std, np.var], margins = True)
    df_pivot['dept'] = str(key)
    #save all the steps
    all_depts_app = all_depts_app.append(df_last)
    all_depts_inspect = all_depts_inspect.append(df_inspect)
    task_filter.update({str(key):filter_posttaskcode['TASK_DESC'].to_list()})
    all_pivot = all_pivot.append(df_pivot)
    print(key)
    
    

fr
pl
pw
sw
ws
ca


In [5]:
task_filter

{'fr': ['CA - U&O Fire Sprinkler Final Approval',
  'Fire Alarm Review On Bldg App',
  'Fire Sprinkler RTC Review - CA Return to Customer',
  'Fire Sprinkler Review On Bldg App',
  'Fire Sprinkler Review on Alarm App',
  'Life Safety Rough In',
  'U&O Alarm Fire Sprinkler Final Approval',
  'U&O Fire Alarm System Final Approval - CA',
  'U&O Fire Alarm System Final Approval - CAAF',
  'U&O Life Safety Final Approval',
  'U&O Underground Work Final Approval - CA',
  '[B]  Fire Alarm System Review',
  '[B]  Fire Sprinkler Review On Bldg App',
  '[B] Fire Alarm Review On Bldg App',
  '[B] Fire Underground Work Review On Bldg App'],
 'pl': [],
 'pw': ['CA - U&O Public Works Final Approval',
  'U&O PW ROW FA - CA Final Approval',
  'U&O PW Ramp & Curb Cut FA - CA Final Approval',
  'U&O PW Ramp RTC & Cut - CA Return to Customer',
  'U&O PW Sidewalk FA - CA Final Approval',
  'U&O PW Solid Waste FA - CA Final Approval'],
 'sw': ['SWGR U&O Sign-off', 'U&O SW Flood Plain Final Approval - CA'],

In [6]:
all_depts_app.RESULT_DESC.value_counts()

Approved                                   128918
Approved with Comments                      48947
No sign-off needed on this project          41071
Issued                                      27004
OK to Pay Permit Fee                        25649
Not Applicable                              25253
Yes                                         12386
No                                           5128
Sidewalks Required                           4689
Paper plans received                         3309
Electronic plans received                    3135
No Capital Project Planned                   1812
Sidewalks Required - In Lieu Elligble        1368
Sidewalks not required                       1279
Allowed and elected to pay in lieu           1217
Elected/Required to construct sidewalks       290
Plans received via email as a PDF             226
Granted with Conditions                       119
BZA Variance Applied for                       85
Zoning Waiver Requested                        77


In [7]:
alltemppivot = pd.pivot_table(all_depts_app, values='deltatask', index=['CASE_TYPE'], aggfunc=[np.mean, np.median, np.min, np.max, np.std, np.var], margins = True)
alltemppivot['dept'] = str('all')
all_pivot = all_pivot.append(alltemppivot)

In [8]:
taskfilterdf = pd.DataFrame({ key:pd.Series(value) for key, value in task_filter.items() })

<ipython-input-8-a55de755209d>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  taskfilterdf = pd.DataFrame({ key:pd.Series(value) for key, value in task_filter.items() })


In [9]:
all_pivot.to_csv('../data/allpivot.csv', index=True)
all_depts_app.to_csv('../data/all_depts_app.csv', index=False)
all_depts_inspect.to_csv('../data/all_depts_inspect.csv', index=False)
taskfilterdf.to_csv('../data/taskfilter.csv', index=False)

In [10]:
all_pivot

,mean,median,amin,amax,std,var,dept
,deltatask,deltatask,deltatask,deltatask,deltatask,deltatask,
CASE_TYPE,,,,,,,
CACA,59.356234,30.0,0.0,529.0,80.371588,6459.592161,fr
CACF,34.920635,15.0,0.0,385.0,58.886835,3467.659286,fr
CACG,58.962264,48.0,16.0,198.0,44.781075,2005.344702,fr
CACH,95.737885,71.0,0.0,604.0,94.584601,8946.246818,fr
CACJ,27.380282,1.0,0.0,152.0,47.071486,2215.724748,fr
...,...,...,...,...,...,...,...
CARP,7.824885,0.0,0.0,124.0,16.301907,265.752174,all
CARR,5.649149,0.0,-30.0,574.0,18.484825,341.688740,all
